# Example JsonSchema

In [1]:
from pprint import pprint
from jsonschema import validate
from json_ntv import Ntv
schema = {"properties" : { "price" : {"type" : "number"}}}
data = {"price": 5}

validate(data, schema)

In [2]:
import pathlib
import json_ntv
from json_ntv.namespace import from_file

file = pathlib.Path(json_ntv.__file__).parent.parent / "RFC" / "NTV_NTVschema_namespace.ini"
from_file(file, '$NTVschema.')

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

## JSONschema

In [ ]:
test = {'family' : 'thomy', 'children': [15, 24, {'simon': None}, {'judith': 22}]}

json_schema = {'properties': {'family':{
                                    'maxLength':10}, #no_type
                              'children':{
                                    'maxItems': 4, #no_type
                                    'items': {
                                        'maximum': 25, 
                                        'propertyNames': {'maxLength':10}}} # no_type, no_name
                              }}
validate(test, json_schema)

`properties` and `prefixItems` are used to navigate in the Json tree

In [ ]:
list_test = [
 ['',                   '',         ''     , 2],
 ['/family',            'family',   'thomy', 1],
 ['/children',          'children', ''     , 4],
 ['/children/0',        '',         15     , 1],
 ['/children/1',        '',         21     , 1],
 ['/children/2',        '',         ''     , 1],
 ['/children/2/simon',  'simon',    None   , 1],
 ['/children/3',        '',         ''     , 1],
 ['/children/3/judith', 'judith',   22     , 1] ]

list_schema = [
 ['/family',     {'maxLength':10}],
 ['/children',   {'maxItems': 4, 'items': {'maximum': 25, 'propertyNames': {'maxLength':10}}}] ]

## NTVschema

In [ ]:
test = {'family:string' : 'thomy', 'children': [{':int': 15}, 21, {'simon':None}, {'judith': 22}]}

schema = {'test::$NTVschema.':
          {'properties.': {'family': {
                                  ':maxLength': 10}, #no_type
                           'children':{  
                                 ':maxItems': 4, #no_type
                                 'items.': {
                                   ':maximum': 25, # no_type, no_name
                                   'name.': {':maxLength': 10}}
                           }}}}
schema = {'test::$NTVschema.':
          {'properties.': {'family': {
                               ':maxLength': 10, #no_type
                               'type.': {':const': 'string'} }, #no_value
                           'children':{  
                               ':maxItems': 4, #no_type
                               'items.': {
                                   ':maximum': 25, # no_type, no_name
                                   'type.': {':enum': ['json', 'int']}, #no_value, no_name
                                   'name.': {':maxLength': 10}},
                               'prefixItems.':[
                                   {'type.': {':const':'int'}}]}}}}

ntv_test = Ntv.obj(test)
ntv_schema = Ntv.obj(schema)

In [ ]:
ntv_test = [
 ['',                   '',         ''     , None    , 2],
 ['/family',            'family',   'thomy', 'string', 1],
 ['/children',          'children', ''     , None    , 4],
 ['/children/0',        '',         15     , 'int'   , 1],
 ['/children/1',        '',         21     , 'json'  , 1],
 ['/children/simon',    'simon',    None   , 'json'  , 1],
 ['/children/judith',   'judith',   22     , 'json'  , 1] ]

list_schema = [
 ['/family',     None,          {':maxLength': 10 }  ], 
 ['/family',     'type.',       {':const': 'string'} ],
 ['/children',   None,          {':maxItems': 4}],
 ['/children',   'items.',      {':maximum': 25} ],
 ['/children',   'items.type.', {':enum': ['json', 'int']} ], 
 ['/children',   'items.name.', {':maxLength': 10} ], 
 ['/children/0', 'type.',       {':const':'int'}] ]

In [ ]:
print(ntv_schema['properties.']['family::type.'])
print(ntv_schema['properties.']['family'])
print(ntv_schema[0][0])
print(ntv_schema['#0/0/0'])
print(ntv_schema['#test/0/0'])
print(ntv_schema['#test/properties./0'])
print(ntv_schema['#test/properties./family'])
print(ntv_schema['#test/properties./family::type.'])

print(ntv_schema[0][2][1])

In [ ]:
#print(ntv_schema.no_name())
#print(ntv_schema.no_type())
pprint(ntv_schema.no_value())
#print(ntv_schema.no_value().no_name())
#print(ntv_schema.no_name().no_type())

In [ ]:
print(ntv_schema.only_type())
print(ntv_schema.only_name())
print(ntv_schema.only_value())

In [ ]:
print(ntv_test.no_name())
print(ntv_test.no_type())
print(ntv_test.no_value())
print(ntv_test.no_value().no_name())
print(ntv_test.no_name().no_type())

In [ ]:
print(ntv_test.only_type())
print(ntv_test.only_name())
print(ntv_test.only_value())

## JsonSchema keywords


### type
Type of a value 

* schema applicable to NTVvalue
```json
{ "type": ["number", "string"] } check if the NTVvalue is a number or a string
```

### String type
length, format, regex 

* schema applicable to NTVvalue 
```json
{ "type": "string", "minLength": 2, "maxLength": 3 } check the length of the NTVvalue
```


### Number type
multiples, range 

* schema applicable to NTVvalue
```json
{ "type": "number", "multipleOf" : 10 } check if the NTVvalue is a multiple of 10
```


### Object : properties
Schema for a value defined by a name 

* schema applicable to the NTVvalue of the NTV entity defined by his relative NTV-pointer 
```json
{ "properties": { "street_name": value_schema } } check the value_schema for the NTVvalue of the NTV entity defined by "street_name" (relative NTV-pointer)
 ```


### Object : patternProperties
Schema for values defined by a name matches a pattern 

* schema applicable to the NTVvalue of the NTV entity whose relative NTV-pointer matches a pattern.
```json
{ "patternProperties": { "^S_": pat_schema } } check the pat_schema for the NTVvalue of the NTV entity whose relative NTV-pointer matches "^S_"
 ```


### Object : additionalProperties
Schema for values whose names are not listed in the properties or patternProperties 

* schema applicable to the NTVvalue of the NTV entity not listed in the properties or patternProperties.
```json
{ "additionalProperties": add_schema } } check the add_schema for the NTVvalue of the NTV entity not listed in the properties or patternProperties.
 ```


### Object : propertyNames
Schema for names 

* schema applicable to the NTVname of the NTV entities 
```json
{ "propertyNames": names_schema } check the names_schema for the NTVnames of the NTV entity 
 ```


### Object : size
Number of properties 

* Number of NTV entities included in the NTV entity
```json
{ "minProperties": 2, "maxProperties": 3 } check the number of NTV entities 
 ```


### Object : other
Required : applicable,

unevaluatedProperties : idem as additionalProperties

### Array : items
Schema for values included in an array 

* schema applicable to the NTVvalue of the NTV entities included in the NTV entity 
```json
{ "items": items_schema } } check the items_schema for the NTVvalue of the NTV entities included in the NTV entity.
 ```


### Array : uniqueItems
Schema to ensure that each of the items in an array is unique 

* schema applicable to the NTVvalue of the NTV entities included in the NTV entity 
```json
{ "uniqueItems": true } } check the uniqueness of the NTVvalue of the NTV entities.
 ```


### Array : prefixItems
Array of schemas for values included in an array 

* schemas applicable to the NTVvalue of the NTV entities included in the NTV entity 
```json
{ "prefixItems": [ item_schema ] } 
check the item_schema for the NTVvalue of the corresponding NTV entity included in the NTV entity.
 ```


### Array : contains, maxContains, minContains
schema for at least one (or number as defined with maxContains or minContains) value included in an array 

* schema applicable to the NTVvalues of the NTV entities included in the NTV entity 
```json
{ "contains": cont_schema, "minContains": 2, "maxContains": 3 } 
check if the cont_schema is valid for 2 or 3 NTVvalue of the NTV entity included.
 ```


### Array : other
unevaluatedItems : applies to any NTVvalues not evaluated by an items, prefixItems, or contains keyword.

minItems, maxItems : equivalent to minProperties, maxProperties


## NTV extension

### xxproperties
"pointer:part": schema

* pointer : relative NTV pointer (integer, NTVname, NTVtype)
* part : name, type or value (default)

### propertyNames 
equivalent to items:name

### required
list of pointers

### xxitems
"xxitems:part": schema

### xxcontains
"xxcontains:part": schema